In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-03-30 11:04:59--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  5.56MB/s    in 0.2s    

2024-03-30 11:04:59 (5.56 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [2]:
with open('input.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()

In [3]:
print(len(text))

1115394


In [4]:
chars = sorted(list(set(text)))
N_vocab = len(chars)
print(''.join(chars))
print(N_vocab)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)}
encode = lambda s : [ stoi[c] for c in s]
decode = lambda l : ''.join([itos[i] for i in l])

In [6]:
import torch
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:30])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43])


In [7]:
N = int(0.9 * len(data))
train_data = data[:N]
valid_data = data[N:]

In [8]:
torch.manual_seed(1337)
N_batch = 4
N_block = 8

def get_batch(split):
    data = train_data if split == 'train' else valid_data
    ix = torch.randint(len(data) - N_block, (N_batch, ))
    x = torch.stack([data[i:i+N_block] for i in ix])
    y = torch.stack([data[i+1:i+N_block+1] for i in ix])
    return x, y
    

## Bigram

In [9]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, N_vocab):
        super().__init__()
        self.token_embedding_table = nn.Embedding(N_vocab, N_vocab)

    def forward(self, idx, targets = None):

        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None

        else:
            B, T, C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim = -1)
            idx_next = torch.multinomial(probs, num_samples = 1)
            idx = torch.cat((idx, idx_next), dim = 1)
        return idx

model = BigramLanguageModel(N_vocab)
xb, yb = get_batch('train')
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)

print(decode(model.generate(idx = torch.zeros((1,1), dtype = torch.long), max_new_tokens = 100)[0].tolist()))

torch.Size([32, 65])
tensor(4.7051, grad_fn=<NllLossBackward0>)

P-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3!dcb


In [10]:
optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-3)

In [11]:
N_batch = 32
for steps in range(10000):
    xb, yb  = get_batch('train')

    logits, loss = model(xb,yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()

print(loss.item())

/home/dsladmin/anaconda3/lib/python3.11/site-packages/torch/autograd/__init__.py:266: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


2.5555150508880615


In [12]:
print(decode(model.generate(idx = torch.zeros((1,1), dtype = torch.long), max_new_tokens = 500)[0].tolist()))


Ong h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercc.
hathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
h hay.JUCle n prids, r loncave w hollular s O:
HIs; ht anjx?

DUThinqunt.

LaZAnde.
athave l.
KEONH:
ARThanco be y,-hedarwnoddy scace, tridesar, wnl'shenous s ls, theresseys
PlorseelapinghiybHen yof GLUCEN t l-t E:
I hisgothers je are!-e!
QLYotouciullle'z,
Th


## Self Attention

In [14]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a,1,keepdim = True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
a,b,c

(tensor([[1.0000, 0.0000, 0.0000],
         [0.5000, 0.5000, 0.0000],
         [0.3333, 0.3333, 0.3333]]),
 tensor([[2., 7.],
         [6., 4.],
         [6., 5.]]),
 tensor([[2.0000, 7.0000],
         [4.0000, 5.5000],
         [4.6667, 5.3333]]))

In [15]:
torch.manual_seed(1337)
B, T, C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [29]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1]
        xbow[b, t] = torch.mean(xprev, 0)

In [30]:
# version 2: using matrix multiply for a weighted aggregation
wt = torch.tril(torch.ones(T,T))
wt = wt / wt.sum(1, keepdim = True)
xbow2 = wt @ x
torch.allclose(xbow, xbow2)

False

In [31]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T,T))
wt = torch.zeros((T,T))
wt = wt.masked_fill(tril == 0, float('-inf'))
wt = F.softmax(wt, dim = -1)
xbow3 = wt @ x
torch.allclose(xbow, xbow3)

False

In [36]:
torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C)

N_head = 16
key = nn.Linear(C, N_head, bias = False)
query = nn.Linear(C, N_head, bias = False)
value = nn.Linear(C, N_head, bias = False)


k = key(x) # B, T, 16
q = query(x) # B,T,16
wt = q @ k.transpose(-2,-1) # B,T,16 @ B,16,T --> B,T,T

print('x',x.shape)
print('k',k.shape)
print('q',q.shape)
print('W',wt.shape)
tril = torch.tril(torch.ones(T,T))
wt = wt.masked_fill(tril == 0, float('-inf'))
wt = F.softmax(wt, dim = -1)

v = value(x)
out = wt @ v

print('O',out.shape)

x torch.Size([4, 8, 32])
k torch.Size([4, 8, 16])
q torch.Size([4, 8, 16])
W torch.Size([4, 8, 8])
O torch.Size([4, 8, 16])


In [37]:
wt[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)